## 說明 

此為TAVI領域之資料

因為SQL語法過長，導致jupyter notebook開啟不順，會當機，故將語法大量刪除，有將跑好的資料存起來，從讀檔開始跑即可

此處有考慮時間段、醫療辭典

In [1]:
#要存model的話 save_model = 1 ; 不存的話 save_model = 0
save_model = 0

## 將SQL資料庫用python查詢

只需將sql語法輸入query = """ 語法 """

因為要跑很久，已經有儲存資料，從下方開始跑!!!

In [2]:
import pandas as pd
import pymssql 
import pickle
'''
conn = pymssql.connect(server="140.112.****.****", user="*****", password="**********", database='***')  
cursor = conn.cursor()
#共2362筆專利
query = """SELECT a.PN
      ,b.[PUBD]
      ,[HEADING]
      ,[CONTENT]
  FROM [Patent].[dbo].[US_GRANT_FT_DESCRIPTION] a
  left join [Patent].[dbo].[US_GRANT_PATENT_INFO] b
  on a.PN =b.PN
where a.pn in('D0841813',
'D0841812',
'D0812226',
'D0732666',
) and HEADING LIKE '%summary%' """
cursor.execute(query)
row = cursor.fetchone()  
'''

'\nconn = pymssql.connect(server="140.112.180.100", user="sa", password="Pris9630", database=\'Patent\')  \ncursor = conn.cursor()\n#共2362筆專利\nquery = """SELECT a.PN\n      ,b.[PUBD]\n      ,[HEADING]\n      ,[CONTENT]\n  FROM [Patent].[dbo].[US_GRANT_FT_DESCRIPTION] a\n  left join [Patent].[dbo].[US_GRANT_PATENT_INFO] b\n  on a.PN =b.PN\nwhere a.pn in(\'D0841813\',\n\'D0841812\',\n\'D0812226\',\n\'D0732666\',\n) and HEADING LIKE \'%summary%\' """\ncursor.execute(query)\nrow = cursor.fetchone()  \n'

In [ ]:
'''
df_original = pd.DataFrame(columns = ['PN', 'CONTENT','PUBD'])
while row:  
    df_original = df_original.append({'PN':str(row[0]), 'CONTENT':str(row[3]), 'PUBD':row[1]}, ignore_index = True)
    #print(str(row[0]) + str(row[1]) + str(row[2]))     
    print(df_original)
    row = cursor.fetchone() 
'''

In [ ]:
#df_original

In [ ]:
#with open('C:/Users/Annie/Desktop/Topic_analysis/patent/50_patents/var_save/df_original_TAVI.pickle', 'wb') as handle:
#    pickle.dump(df_original, handle, protocol=pickle.HIGHEST_PROTOCOL)

# -----從這開始跑-----

In [ ]:
with open('var_save/df_original_TAVI.pickle', 'rb') as handle:
    a = pickle.load(handle)

In [ ]:
df_original

In [ ]:
print(len(df_original.PN.unique #2362 -> 2264
df_original.PN.unique()

## ---這段也可以不用跑，有存df_summary---

In [ ]:
df_original

In [ ]:
print(type(df_original['PUBD'][0]))

In [ ]:
'''
df_summary = pd.DataFrame(columns = ['PN', 'CONTENT', 'YEAR'])
for pn in df_original['PN'].unique():
    content = ''
    for i in range(len(df_original)):
        if df_original['PN'][i] == pn:
            year = df_original['PUBD'][i]
            content += df_original['CONTENT'][i]
    #print(pn,year, '\n' ,content)
    df_summary = df_summary.append({'PN':pn, 'CONTENT':content,'YEAR':year}, ignore_index = True)
    # df_original = df_original.append({'PN':str(row[1]), 'CONTENT':str(row[4]), 'DATE':row[7]}, ignore_index = True)
'''

In [ ]:
#df_summary

In [ ]:
'''
import pickle
with open('C:/Users/Annie/Desktop/Topic_analysis/patent/50_patents/var_save/df_summary_TAVI.pickle', 'wb') as handle:
    pickle.dump(df_summary, handle, protocol=pickle.HIGHEST_PROTOCOL)
'''

## -----------------------從這開始繼續!---------------------------

In [3]:
with open('var_save/df_summary_TAVI.pickle', 'rb') as handle:
    df_summary = pickle.load(handle)

In [4]:
df_summary

,PN,CONTENT,YEAR
0,07166570,"Briefly stated, the present invention provides...",2007-01-23
1,07175656,The present invention provides improved device...,2007-02-13
2,07195640,The medical devices in combination with therap...,2007-03-27
3,07241295,It is an object of the invention to provide a ...,2007-07-10
4,07261732,"Accordingly, an object of the present inventio...",2007-08-28
...,...,...,...
2259,10849745,Embodiments hereof relate to a balloon cathete...,2020-12-01
2260,08038710,The present invention provides artificial valv...,2011-10-18
2261,10729542,"As discussed above, stented prosthetic heart v...",2020-08-04
2262,09801721,"In some embodiments, a sizing device for use i...",2017-10-31


## 新增手動查詢的summary

如果沒有人工查詢其他summary資料，可以不用執行

In [ ]:
#有手動新增資料 == 1 ;沒有 == 0
add_summary = 0

In [ ]:
if add_summary == 1:
    df_loss = pd.read_excel('DESCRIPTION_SUMMARY_0521.xlsx','工作表1')  #增加手動新增的summary，注意格式要PN / summary / PUBD
    documents_merge = df_summary.append(df_loss,ignore_index = True)
else:
    documents_merge = df_summary

documents_merge['YEAR'] = pd.to_datetime(documents_merge['YEAR']).dt.date #不顯示時間

In [ ]:
documents_merge

## 3 period

In [ ]:
from datetime import date

In [ ]:
year06_11 = []
year11_16 = []
year16_21 = []
for i in range(len(documents_merge.YEAR)):
    if documents_merge.iloc[i][2] <= date(2010, 12, 31):
        year06_11.append(str(documents_merge.iloc[i][1]))
    elif documents_merge.iloc[i][2] <= date(2015, 12, 31):
        year11_16.append(str(documents_merge.iloc[i][1]))
    elif documents_merge.iloc[i][2] <= date(2021, 12, 31):
        year16_21.append(str(documents_merge.iloc[i][1]))

In [ ]:
print(len(year06_11), len(year11_16), len(year16_21))
#13 -> 12
#18 -> 13 -> 15
#19 -> 14 -> 16

In [ ]:
df_1 = pd.DataFrame(year06_11, columns = ['summary'])
df_2 = pd.DataFrame(year11_16, columns = ['summary'])
df_3 = pd.DataFrame(year16_21, columns = ['summary'])

In [ ]:
df_1

In [ ]:
df_2

In [ ]:
df_3

## MedTerm

是否要用UMLS!????????????????→ 先不用

In [ ]:
f = open('MedTerm.txt')
text = []
MedTerm = []
for line in f:
    text.append(line)
for i in text:
    MedTerm.append(i.split('\n', 1)[0].lower())

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

In [ ]:
#lemmatize_stemming('amini')

In [ ]:
min_len = 1

In [ ]:
def lemmatize_stemming(text):
    # 把字詞幹化
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess_term(text):
    term_lemma = []
    term_tmp = []
    result = [] 
    #print(text)
    
    for token in gensim.utils.simple_preprocess(text, min_len=min_len, max_len=35):  #tokenize
        #print(token)
        term_tmp.append((lemmatize_stemming(token)))
    term_lemma.append(' '.join(term_tmp))
    return term_lemma

In [ ]:
MedTerm_lemma = []  #儲存詞幹化之後的MedTerms
for w in MedTerm:
    #print(preprocess(w))
    MedTerm_lemma.append(preprocess_term(w))

In [ ]:
MedTerm_lemma_fin = []   #將MedTerms弄成跟原本的MedTerms一樣格式
for i in range(len(MedTerm_lemma)):
    for j in range(len(MedTerm_lemma[i])):
        MedTerm_lemma_fin.append(MedTerm_lemma[i][0])

In [ ]:
 # import MWETokenizer() method from nltk 
from nltk.tokenize import MWETokenizer 
   
# Create a reference variable for Class MWETokenizer  

MedTerm_mwe = []  #將兩個以上單字組成的字詞用tuple刮在一起
#tmp = ()
for i in range(len(MedTerm_lemma_fin)):
    if len(MedTerm_lemma_fin[i].split(' '))> 1:
        tmp = ()
        tmp = tmp + tuple(MedTerm_lemma_fin[i].split(' '))
        MedTerm_mwe.append(tmp)
    else:
        MedTerm_mwe.append(MedTerm_lemma_fin[i])

In [ ]:
#兩個以上的字用_連接
tk = MWETokenizer(MedTerm_mwe) 
MedTerm_token = []
MedTerm_token_show = []  #將長度=0的也留著(為了方便表格觀看)
for i in range(len(MedTerm_lemma_fin)):
    if len(MedTerm_lemma_fin[i]) != 0:
        MedTerm_token.append(tk.tokenize(MedTerm_lemma_fin[i].split())[0])
        MedTerm_token_show.append(tk.tokenize(MedTerm_lemma_fin[i].split())[0])       
    else:
        MedTerm_token_show.append('')
    #print(token_medterm[0])

In [ ]:
for token in MedTerm_token:
    print(token)

## Data pre-processing

In [ ]:
def lemmatize_stemming(text):
    # 把字詞幹化
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    docu_tmp = []
    documents_lemma = []
    result = [] 
    stop_word = ['ul', 'id', 'sub']
    for token in gensim.utils.simple_preprocess(text, min_len=min_len, max_len=35):  #tokenize
        docu_tmp.append(lemmatize_stemming(token))
    documents_lemma.append(' '.join(docu_tmp))

    text_token = tk.tokenize(documents_lemma[0].split())
    for token in text_token:
        if token in MedTerm_token and token not in gensim.parsing.preprocessing.STOPWORDS and token not in stop_word:
            result.append(token)
    return result

In [ ]:
# 分三個年份
processed_docs_1 = df_1['summary'].map(preprocess)
processed_docs_2 = df_2['summary'].map(preprocess)
processed_docs_3 = df_3['summary'].map(preprocess)

In [ ]:
# 不分年分
processed_docs = documents_merge['CONTENT'].map(preprocess)

## Bag of Words on the Data set

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
for k, v in dictionary.iteritems():
    print(v)

In [ ]:
print(dictionary)

In [ ]:
dictionary_1 = gensim.corpora.Dictionary(processed_docs_1)
dictionary_2 = gensim.corpora.Dictionary(processed_docs_2)
dictionary_3 = gensim.corpora.Dictionary(processed_docs_3)

In [ ]:
print(dictionary_1)
print(dictionary_2)
print(dictionary_3)

In [ ]:
for k, v in dictionary_1.iteritems():
    print(v)

In [ ]:
for k, v in dictionary_2.iteritems():
    print(v)

In [ ]:
for k, v in dictionary_3.iteritems():
    print(v)

In [ ]:
dictionary_1.filter_extremes(no_below=0, no_above=0.5, keep_n=100000)
dictionary_2.filter_extremes(no_below=0, no_above=0.5, keep_n=100000)
dictionary_3.filter_extremes(no_below=0, no_above=0.5, keep_n=100000)

In [ ]:
print(dictionary_1)
print(dictionary_2)
print(dictionary_3)

## Gensim doc2bow

In [ ]:
bow_corpus_1 = [dictionary_1.doc2bow(doc) for doc in processed_docs_1]
bow_corpus_2 = [dictionary_2.doc2bow(doc) for doc in processed_docs_2]
bow_corpus_3 = [dictionary_3.doc2bow(doc) for doc in processed_docs_3]

In [ ]:
#processed_docs = documents_merge['TACD'].map(preprocess)

In [ ]:
#processed_docs

## Bag of Words on the Data set

In [ ]:
#dictionary = gensim.corpora.Dictionary(processed_docs)
#print(dictionary)

In [ ]:
#dictionary.filter_extremes(no_below=3, no_above=0.5, keep_n=100000)
#print(dictionary)

## Gensim doc2bow

In [ ]:
#bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

## Find the optimal number of topics for LDA?

In [ ]:
from gensim.models import CoherenceModel
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=50, workers=8)
        #model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list_1, coherence_values_1 = compute_coherence_values(dictionary=dictionary_1, corpus=bow_corpus_1, texts=processed_docs_1, start=2, limit=9, step=1)
model_list_2, coherence_values_2 = compute_coherence_values(dictionary=dictionary_2, corpus=bow_corpus_2, texts=processed_docs_2, start=2, limit=9, step=1)
model_list_3, coherence_values_3 = compute_coherence_values(dictionary=dictionary_3, corpus=bow_corpus_3, texts=processed_docs_3, start=2, limit=9, step=1)

In [ ]:
# Show graph
import matplotlib.pyplot as plt
%matplotlib inline

limit=9; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values_1)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

x = range(start, limit, step)
plt.plot(x, coherence_values_2)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

x = range(start, limit, step)
plt.plot(x, coherence_values_3)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values_1):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4)) #2
    
for m, cv in zip(x, coherence_values_2):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4)) #3

for m, cv in zip(x, coherence_values_3):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4)) #4

In [ ]:
# Select the model and print the topics
from pprint import pprint
optimal_model_1 = model_list_1[3]
model_topics = optimal_model_1.show_topics(formatted=False)
#pprint(optimal_model.print_topics(num_words=10))

optimal_model_2 = model_list_2[3]
model_topics = optimal_model_2.show_topics(formatted=False)
#pprint(optimal_model.print_topics(num_words=10))

optimal_model_3 = model_list_3[-3]
model_topics = optimal_model_3.show_topics(formatted=False)
#pprint(optimal_model.print_topics(num_words=10))

## save model

In [ ]:
from gensim.corpora import MmCorpus
if save_model == 1:
    MmCorpus.serialize('patent_1.mm', bow_corpus_1)
    dictionary_1.save('patent_1.dict')
    MmCorpus.serialize('patent_2.mm', bow_corpus_2)
    dictionary_2.save('patent_2.dict')
    MmCorpus.serialize('patent_3.mm', bow_corpus_3)
    dictionary_3.save('patent_3.dict')
    optimal_model_1.save('patent_1.model')
    optimal_model_2.save('patent_2.model')
    optimal_model_3.save('patent_3.model')

## Visualize the LDA model

In [ ]:
#lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=100, workers=8)

In [ ]:
#coherencemodel = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
#coherencemodel.get_coherence()

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data_1 = gensimvis.prepare(topic_model=optimal_model_1, corpus=bow_corpus_1, dictionary=optimal_model_1.id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data_1)

In [ ]:
vis_data_2 = gensimvis.prepare(topic_model=optimal_model_2, corpus=bow_corpus_2, dictionary=optimal_model_2.id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data_2)

In [ ]:
vis_data_3 = gensimvis.prepare(topic_model=optimal_model_3, corpus=bow_corpus_3, dictionary=optimal_model_3.id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data_3)